In [91]:
import numpy as np
import open3d as o3d
from scipy.spatial.transform import Rotation
N = 100

In [92]:
# Sample 50 points in x-z plane in range (5, 3)
x_samples = np.random.uniform(low=0, high=7, size=(N, 1))
z_samples = np.random.uniform(low=0, high=4, size=(N, 1))
y_samples = np.zeros((N, 1))
gt_points = np.hstack((x_samples, y_samples, z_samples))

# Visualize the sampled points
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(gt_points)
pcd.paint_uniform_color(np.array([[0, 1, 0]]).T)

# Visualize the mean x-z plane segment
gt_plane = o3d.geometry.TriangleMesh.create_box(width=7, height=0.01, depth=4)
gt_plane.paint_uniform_color(np.array([[0, 1, 0]]).T)
coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame()
o3d.visualization.draw_geometries([pcd, gt_plane, coord_frame])

In [93]:
noisy_planes = []

for i in range(1):
    # Add gaussian noise to the points in y_direction
#     y_noise = np.random.normal(0, 0.5, size=(N, 1))
#     x_noise = 1.5*np.random.random((N, 1)) - 0.75
#     y_noise = 1.5*np.random.random((N, 1)) - 0.75
#     z_noise = 1.5*np.random.random((N, 1)) - 0.75
    x_noise = np.random.normal(0, 0.5, size=(N, 1))
    y_noise = np.random.normal(0, 0.5, size=(N, 1))
    z_noise = np.random.normal(0, 0.5, size=(N, 1))
    noisy_points = np.hstack((x_samples+x_noise, y_samples+y_noise, z_samples+z_noise))

    # Visualize the sampled points
    noisy_pcd = o3d.geometry.PointCloud()
    noisy_pcd.points = o3d.utility.Vector3dVector(noisy_points)
    noisy_pcd.paint_uniform_color(np.array([[1, 0, 0]]).T)
    obox = noisy_pcd.get_oriented_bounding_box()
    abox = noisy_pcd.get_axis_aligned_bounding_box()
#     noisy_planes.append(obox)
#     o3d.visualization.draw_geometries([noisy_pcd, obox, pcd, coord_frame])

    plane_params, some_list = noisy_pcd.segment_plane(1.0, int(N), 5)
    plane_params = plane_params/np.linalg.norm(plane_params[:3])
    print(plane_params)

    """ Draw mean plane of noisy point cloud """
    gt_normal = np.array([[0, 1, 0]])
    noisy_normal = plane_params[:3].reshape((1, 3))
    # Find the relative rotation between normals
    R, error = Rotation.align_vectors(gt_normal, noisy_normal)
    R.as_matrix(), error

    # Find the nearest point on the plane to origin
    l = -plane_params[-1]
    noisy_origin = (l*plane_params[:3]).reshape((3, 1))
    # noisy_origin

    # Compute the relative transformation w.r.t ground truth plane segment
    noisy_plane = o3d.geometry.TriangleMesh.create_box(width=5, height=0.01, depth=3)
    noisy_plane.paint_uniform_color(np.array([[0, 0, 1]]).T)
#     noisy_plane.translate(-R.as_matrix().T@noisy_origin)
#     obox.translate(-R.as_matrix().T@noisy_origin)
#     obox.rotate(R.as_matrix().T)
    noisy_planes.append(abox)

# Create a plane segment box and transform to estimated segment
o3d.visualization.draw_geometries([pcd, gt_plane, coord_frame] + noisy_planes)

[-0.01179844  0.97437197 -0.22463315  0.33878613]


/home/tvvsstas/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.


In [94]:
# # Create a test point cloud
# x = np.arange(0, 1, 0.05)
# y = np.arange(0, 2, 0.05)
# z = np.arange(0, 3, 0.05)

# xs, ys, zs = np.meshgrid(x, y, z)
# pts = np.vstack((xs.flatten(), ys.flatten(), zs.flatten())).T
# pts_ = np.hstack((pts, np.ones((pts.shape[0], 1))))

# R = np.array([[0, 1, 0], [0, 0, -1], [-1, 0, 0]]) # is inverse rotation
# t = np.array([[-1, -2, -3]]).T
# w2c = np.hstack((R, t))
# w2c = np.vstack((w2c, np.array([[0, 0, 0, 1]]))) # transforms from w to c

# tp = o3d.geometry.PointCloud()
# # tp.points = o3d.utility.Vector3dVector((R.T@pts.T-R.T@t).T)
# tp.points = o3d.utility.Vector3dVector(((R@(pts.T+t)).T)/5.0)
# # tp.points = o3d.utility.Vector3dVector(pts)
# coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame()
# o3d.visualization.draw_geometries([tp, coord_frame])

In [101]:
def get_plane_offset(plane_model, points, inliers):
    points = np.hstack((points, np.ones((points.shape[0], 1))))
    num_of_inliers = len(inliers)
    
    pp_sum = 0.0
    pp_squares_sum = 0.0
    centroid = np.zeros(3)
    for i in range(num_of_inliers):
        point = points[inliers[i], :]
        pp_distance = point.flatten().dot(plane_model.flatten())
        centroid += points[inliers[i], :3].flatten()
        
        pp_sum += pp_distance
        pp_squares_sum += pp_distance**2
    
    mean = pp_sum/len(inliers)
    variance = (pp_squares_sum - ((pp_sum**2)/num_of_inliers)) / (num_of_inliers-1)
    std_dev = variance**0.5
    centroid = centroid/num_of_inliers

    """ 
    For a Gaussian distribution
    68% values lie within 1*std_dev
    95% values lie within 2*std_dev
    99.7% values lie within 3*std_dev
    """
    return 2*std_dev, centroid

# Fit a plane fixing the normal
plane_normal = np.array([[0, 1, 0]]).T
plane_params, inliers = noisy_pcd.segment_plane(0.5, int(N/10), 100)
plane_params[:3] = plane_normal.flatten()

# Find min and max 'd' values
delta, centroid = get_plane_offset(plane_params, noisy_points, inliers)

# Compute bounding box dimensions from plane delta and centroid
plane_normal = np.array([[1, delta, 1]]).T
box_points = np.array([[0, delta, 0], [0, -delta, 0]])
box_points = np.vstack((gt_points, box_points))

bbox = o3d.geometry.PointCloud()
bbox.points = o3d.utility.Vector3dVector(box_points)

noisy_plane = o3d.geometry.TriangleMesh.create_box(width=7, height=0.01, depth=4)
noisy_plane.paint_uniform_color(np.array([[0, 0, 1]]).T)
noisy_plane.translate(np.array([[0, centroid[1], 0]]).T)

# Create a plane segment box and transform to estimated segment
o3d.visualization.draw_geometries([bbox.get_axis_aligned_bounding_box(), noisy_pcd, coord_frame, gt_plane, noisy_plane])
print(delta)

0.7622766450415077
